In [1]:
import locale
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import numpy as np
from scipy.optimize import curve_fit

locale.setlocale(locale.LC_ALL, "en_US")

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1Wd-p3P46K7ERwMMmCFpX8ZyD-VV9X8t5E2kDts3qs-w'

In [2]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

sheet = service.spreadsheets()

def load_data():
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range='Data!A2:I').execute()
    values = result.get('values', [])
    for i, row in enumerate(values):
        for j, val in enumerate(row):
            if '%' in val:
                values[i][j] = locale.atof(val.rstrip("%")) / 100
            else:
                values[i][j] = locale.atof(val)
    values = np.array(values, dtype=float)
    return (values[:, i] for i in range(9))

def store_data(sol, range, winf):
    values = [winf] + list(sol)
    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=range,
                          body={'values': values}).execute()

In [3]:
dist, _, elev, temp, weight, wind, _, power, speed = load_data()

In [4]:
def f(x, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13):
    d, e, t, w, n, p = x
    return c1 + c2*d + c3*d**2 + c4*e + c5*e**2 + c6*t + c7*t**2 + c8*w + c9*w**2 + c10*n + c11*n**2 + c12*p + c13*p**2

In [5]:
sol, _ = curve_fit(f, (dist, elev, temp, weight, wind, power), speed)
print("c1 = {:g}, c2 = {:g}, c3 = {:g}, c4 = {:g}, c5 = {:g}, c6 = {:g}, c7 = {:g}, c8 = {:g}, c9 = {:g}, c10 = {:g}, c11 = {:g}, c12 = {:g}, c13 = {:g}".format(*sol))
#store_data(sol, 'Data!AB2')

c1 = 80.7452, c2 = 0.0402536, c3 = -0.000438487, c4 = -357.908, c5 = 1481.72, c6 = 0.0912924, c7 = -0.00059398, c8 = -0.555713, c9 = 0.00101737, c10 = -0.14662, c11 = 0.00930723, c12 = 6.72189, c13 = -0.762562
